In [1]:
import os
os.chdir('../')
os.environ["TRANSFORMERS_CACHE"]="/common/users/yg334/LLAMA/huggingface/cache/"

from sentence_transformers import SentenceTransformer, util
from general_dataset import GeneralDataset
from torch.utils.data import DataLoader
import torch
import openai
import numpy as np
from IPython.utils import io
from agi_utils import *
import random
from tqdm import tqdm
from evaluate import load
from torchvision import transforms
from transformers import AutoModel, AutoFeatureExtractor
from torchmetrics.multimodal import CLIPScore
from combine_model_seq import SeqCombine

/common/home/yg334/anaconda3/envs/claude/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /common/home/yg334/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /common/home/yg334/anaconda3/envs/claude/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


In [2]:
"""
assign openagi data path 
"""
# data_path = "YOUR_DATA_PATH"
data_path = "/common/users/yg334/openagi_data/"

task_discriptions = txt_loader("./task_description.txt")
# test_task_idx = [0,21,61,105,110,120,10,35,62,107,115]
test_task_idx = [2,3,10,15,20,35,45,55,65,70,70,90,106,107]
# test_task_idx = list(range(183))
test_dataloaders = []
for i in tqdm(test_task_idx):
    dataset = GeneralDataset(i, data_path)
    dataloader = DataLoader(dataset, batch_size=5)
    test_dataloaders.append(dataloader)
    
test_tasks = [task_discriptions[i].strip() for i in test_task_idx]

100%|███████████████████████████████████████████| 14/14 [00:03<00:00,  4.11it/s]


In [3]:
# device_list = ["cuda:1","cuda:2","cuda:3","cuda:4","cuda:5","cuda:7","cpu"]
device_list = ["cuda:0","cuda:2","cpu"]
eval_device = "cuda:1"
clip_score = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

# Load a pre-trained Vision Transformer model and its feature extractor
vit_ckpt = "nateraw/vit-base-beans"
vit = AutoModel.from_pretrained(vit_ckpt,cache_dir="/common/users/yg334/LLAMA/huggingface/cache/")
vit.eval()
vit_extractor = AutoFeatureExtractor.from_pretrained(vit_ckpt,cache_dir="/common/users/yg334/LLAMA/huggingface/cache/")

f = transforms.ToPILImage()
bertscore = load("bertscore")

sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")

seqCombination = SeqCombine(device_list)

Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [4]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

api_key = "YOUR TOKEN"
anthropic = Anthropic(
    api_key=api_key,
)

In [12]:
import openai

openai.api_key = "YOUR TOKEN"

def generate_module_list_with_gpt(generated_module_seq):
    todo_prompt = "You are a key phrase extractor who is able to extract potential module names from the given context. You have already known all the module names in the full module list. The full module list is: [Image Classification, Colorization, Object Detection, Image Deblurring, Image Denoising, Image Super Resolution, Image Captioning, Text to Image Generation, Visual Question Answering, Sentiment Analysis, Question Answering, Text Summarization, Machine Translation]. Given the following context: '{}'. Please extract a module sequence from this context and remove module names which do not exist in the full module list from this sequence. Output the module sequence after filtering as the format of 'module: module1, module: module2, module: module3, etc...'. "
    prompt = todo_prompt.format(generated_module_seq)

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message["content"]
    
    # print(content)
    
    content = content.split("module: ")[1:]
    
    result = ""
    for c in content:
        result += c
    
    # result = result[:-1] if len(result) > 0 else result
    
    return result

# generated_module_list = generate_module_list_with_gpt(response[prompt_length:])
# print(generated_module_list)

In [14]:
rewards = []
clips = []
berts = []
similairies = []

# eval_device = "cuda:5"

task_len = len(test_tasks)

for i, task_description in enumerate(tqdm(test_tasks)):
    task_description = test_tasks[i]
    print(task_description)
    task_rewards = []
    with torch.no_grad():
        completion = anthropic.completions.create(
                    model="claude-2",
                    max_tokens_to_sample=3000,
                    prompt=f"{HUMAN_PROMPT} {task_description} {AI_PROMPT}",
                )
        module_list = generate_module_list_with_gpt(completion.completion)
        print(module_list)
    

    if len(module_list) > 0 and whole_module_seq_filter(module_list, test_task_idx[i]):
        seqCombination.construct_module_seq(module_list)

        for idx, batch in enumerate(test_dataloaders[i]):
            inputs = list(batch['input'][0])
            predictions = seqCombination.run_module_seq(inputs)
            
            if 0 <= test_task_idx[i] <= 14:
                outputs = list(batch['output'][0])
                dist = image_similarity(predictions, outputs, vit, vit_extractor)
                task_rewards.append(dist / 100)
            elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
                outputs = list(batch['output'][0])
                f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=eval_device))
                
                task_rewards.append(f1)
            else:
                score = clip_score(predictions, inputs)
                task_rewards.append(score.detach()/100)
                
        ave_task_reward = np.mean(task_rewards)    
        seqCombination.close_module_seq()
            
    else:
        ave_task_reward = 0
        
    print(ave_task_reward)
        
    if 0 <= test_task_idx[i] <= 14:
        similairies.append(ave_task_reward)
    elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
        berts.append(ave_task_reward)
    else:
        clips.append(ave_task_reward)

    rewards.append(ave_task_reward)     
    

print("Finished testing!")    

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Given low-resolutioned blurry grayscale image, how to return the regular image step by step?
Image Super Resolution, Image Denoising, Colorization


  7%|███                                        | 1/14 [01:49<23:39, 109.19s/it]

Given blurry grayscale image, how to return the regular image step by step?
Image Deblurring, Image Denoising, Image Super Resolution


 14%|██████▎                                     | 2/14 [03:13<18:54, 94.58s/it]

...

Given English text, how to generate a image step by step?


 93%|███████████████████████████████████████▉   | 13/14 [10:52<00:45, 45.86s/it]

Text to Image Generation, Visual Question Answering, Image Super Resolution
Given clozed English text, how to return the summarization in German step by step?
Text Summarization, Machine Translation


100%|███████████████████████████████████████████| 14/14 [11:53<00:00, 50.97s/it]

Finished testing!


In [17]:
np.mean(clips), np.mean(berts), np.mean(similairies), (np.mean(clips)+np.mean(berts)+np.mean(similairies))/3

(0.0, 0.22933686961233613, 0.6858464152018229, 0.305061094938053)